<a href="https://colab.research.google.com/github/ranvirsahota/AiCore/blob/14-advanced-sql/11_types_of_subqueries/notebook_lesson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import sys
sys.path.append('/home/ranvir/AiCore')
from database_connection import DatabaseConnection
db = DatabaseConnection()


<h1 style="color: rgb(241, 90, 36)"><img src="https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/14.%20Types%20of%20Subqueries/images/SQLIcon.png?modified=23223&raw=1" width=80px height=80px style="vertical-align: middle;"> Types of Subqueries</h2>

There are various types of subqueries that can be used to get the results you want.

<h3 style="color: rgb(241, 90, 36)"> Single row and scalar subqueries</h3>

These types of subqueries either return a **single row** or *scalar* value. Some common uses for them are:

- Updating, deleting or inserting data based on the row returned by the subquery
- Returning a **scalar** value which can be used as a comparison in your `HAVING` and `WHERE` statements
- Setting a default value for columns in your `SELECT` statement in the case of a `NULL`
- Updating a single value in your table

Let's look at some examples. Imagine we wanted to get all films from the `film` table which have a greater than average `runtime`. We could do so with the following query:

>All example code will be run on the **pagila** database, so make sure you have it available to run the code in this notebook.

In [5]:
db.execute_query('''
                    SELECT title,
                        length
                    FROM
                        film
                    WHERE
                        length >
                        (
                            SELECT AVG(length)
                            FROM film
                        )
                    ORDER BY
                        length DESC;
                ''')

,title,length
0,DARN FORRESTER,185
1,WORST BANGER,185
2,MUSCLE BRIGHT,185
3,CHICAGO NORTH,185
4,CONTROL ANTHEM,185
...,...,...
484,RESURRECTION SILVERADO,117
485,WORDS HUNTER,116
486,DUCK RACER,116
487,INSTINCT AIRPORT,116


Which returns all movies with a `length` greater than the **average** runtime, note here our subquery:

In [6]:
db.execute_query('''
                    (
                        SELECT AVG(length)
                        FROM film
                    );
                ''')

,avg
0,115.272


Returns a scalar value `115.272`. Since subqueries are evaluated inside out, imagine the `WHERE` statement is evaluated, then it reads:

`WHERE length > 115.272`

So it's filtering the table for rows where `length > 115.272`. All the usual **comparison operators** would have worked here for a scalar subquery `<, >, =`, `>=`, `<=` or `<>`. Ordering the data we get the following result:

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/14.%20Types%20of%20Subqueries/images/average_length_subquery.png?modified=343232&raw=1'>

Now what if we wanted to display the **average length** alongside the `length` column to visually validate our data. Then we could actually create a new column based on a subquery inside the `SELECT` statement.

In [2]:
db.execute_query('''
                    SELECT title,
                        length,
                        (
                                SELECT ROUND(AVG(length)) AS average_film_length
                                FROM film
                        )
                    FROM
                        film
                    WHERE
                        length >
                        (
                            SELECT AVG(length)
                            FROM film
                        )
                    ORDER BY
                        length;
                ''')

,title,length,average_film_length
0,WORDS HUNTER,116,115.0
1,INSTINCT AIRPORT,116,115.0
2,DUCK RACER,116,115.0
3,MADIGAN DORADO,116,115.0
4,AFFAIR PREJUDICE,117,115.0
...,...,...,...
484,SWEET BROTHERHOOD,185,115.0
485,MUSCLE BRIGHT,185,115.0
486,POND SEATTLE,185,115.0
487,CONTROL ANTHEM,185,115.0


Returning the results:

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/14.%20Types%20of%20Subqueries/images/scalar_sub_select.png?modified=32232&raw=1'>

The `length` column has been sorted in **descending** order this time, to show that all movies in the resultant query are in fact over the **average** length.

> So we can actually use subqueries in `SELECT` statements to generate new columns if they return a scalar value.

We can also use a subquery in the `HAVING` statement to filter the data:

In [3]:
db.execute_query('''
                    SELECT AVG(length) AS film_rating_average_length,
                        rating
                    FROM
                        film
                    GROUP BY
                        rating
                    HAVING
                        AVG(length) >
                        (
                            SELECT AVG(length)
                            FROM film
                        );
                ''')

,film_rating_average_length,rating
0,120.443946,PG-13
1,118.661538,R


Here we've filtered the results after the `GROUP BY` statement using the `HAVING` statement. Let's group the films by `rating` and then determine which `rating`s have a higher than average `replacement_cost`.

In [7]:
db.execute_query('''
                    SELECT rating,
                        ROUND(AVG(replacement_cost), 2) AS avg_rating_replacement_cost,
                        (
                                SELECT ROUND(AVG(replacement_cost),2) AS avg_film_replacement_cost
                                FROM film
                        )
                    FROM
                        film
                    GROUP BY
                        rating
                    HAVING
                        AVG(replacement_cost) >
                        (
                            SELECT AVG(replacement_cost)
                            FROM film
                        );
                ''')

,rating,avg_rating_replacement_cost,avg_film_replacement_cost
0,PG-13,20.40,19.98
1,NC-17,20.14,19.98
2,G,20.12,19.98
3,R,20.23,19.98


We've used another subquery to create a column with the average `replacement_cost` of all movies, so we can validate the data. In the `HAVING` statement we're checking that the average `replacement_cost` for that specific rating is greater than the average `replacement_cost` across all movies.

<h3 style="color: rgb(241, 90, 36)"> Multiple row subquery</h3>

Subqueries that return multiple rows of data to an **outside query** are called *multiple-row subqueries*. These are great for targeting a **subsets** of the data. They can be used in the `SELECT`, `FROM`, `WHERE` and `HAVING` just like *single row subqueries*. They can't be used with **comparison operators**, unless preceded with one of the **keywords** `ANY`, `ALL`, `IN` or `NOT IN`.

Let's imagine we want to check which actors appeared in the film `ACE GOLDFINGER` in the `film` table, which has `film_id = 2`. We could do this by joining the two tables and then filtering the data in the `WHERE` statement. Or we could use a subquery to get the data:


In [ ]:
SELECT actor_id,
       first_name,
       last_name
FROM
    actor
WHERE
    actor_id IN
    (
        SELECT actor_id
        FROM film_actor
        WHERE film_id = 2
    );

First we evaluate the inner query which selects all rows from the `film_actor` table where the `film_id = 2`. This should give us a **subset** of the `film_actor` table since there are multiple rows meeting the condition. You can check this by running the subquery as its own query, it should produce the following result:



<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/14.%20Types%20of%20Subqueries/images/multi_row_subquery_run.png?modified=234324&raw=1'>

So after the subquery is evaluated, the `WHERE` statement would then become `WHERE actor_id IN (19, 85, 90, 160)` which allows us to produce the final result.

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/14.%20Types%20of%20Subqueries/images/subset_filter_multi_row.png?modified=23424&raw=1'>

Ideally, we would like to `SELECT` the `film_id` column to be part of the final result, so we can verify the accuracy of the results. Unfortunately, the `film_id` column exists in the `film_actor` table, which the subquery selects from and does not exist in `actor` table used by the **outer query**.

<h4 style="color: rgb(241, 90, 36)"> ANY, ALL, IN, NOT IN</h4>

`ANY`, `ALL`, `IN`, `NOT IN` can be used in conjunction with a **comparison operator**. In all cases the subquery must return **at least one column** to be used with these subquery expressions. They have the following use cases:

- <b style="color: rgb(241, 90, 36)">ANY/SOME</b>: `ANY` or `SOME` can be used interchangeably. If the evaluated is `True` for any row returned by the subquery, then the statement will ultimately evaluate to `True`.
- <b style="color: rgb(241, 90, 36)">IN</b>: Evaluates to `True` if the expression matches any rows in the subquery
- <b style="color: rgb(241, 90, 36)">NOT IN</b>: Evaluates to `True` if the expression does not match any rows in the subquery
- <b style="color: rgb(241, 90, 36)">ALL</b>: The statement will evaluate to `True` if the expression matches all rows in the subquery


In [ ]:
{expression} {operator} [ ALL | ANY | IN | NOT IN] (subquery)

Let's take a look at some examples:

#### `IN` Subquery

In [38]:
db.execute_query('''
                    SELECT title,
                        release_year,
                        replacement_cost
                    FROM
                        film
                    WHERE
                        replacement_cost IN
                        (
                            SELECT DISTINCT(replacement_cost)
                            FROM film
                            WHERE replacement_cost < 12.99
                            AND replacement_cost > 9.99
                            )
                    ORDER BY
                            title ASC
                ''').tail(10)
# ADDED THE 'ORDER BY' for comparing results against other queries

,title,release_year,replacement_cost
88,UNTOUCHABLES SUNRISE,2006,11.99
89,VAMPIRE WHALE,2006,11.99
90,VIDEOTAPE ARSENIC,2006,10.99
91,WARLOCK WEREWOLF,2006,10.99
92,WESTWARD SEABISCUIT,2006,11.99
93,WHALE BIKINI,2006,11.99
94,WITCHES PANIC,2006,10.99
95,WRONG BEHAVIOR,2006,10.99
96,YENTL IDAHO,2006,11.99
97,ZHIVAGO CORE,2006,10.99


Here the subquery gets all movies with `replacement_cost` < `12.99`. If we run it on its own we get the following result:

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/14.%20Types%20of%20Subqueries/images/replacement_cost_sub.png?modified=2234&raw=1'>

In the query we're using the `IN` keyword to return all movies which have one of these rental ratings.

#### `> ANY` Subquery

In [37]:
db.execute_query('''
                    SELECT title,
                        release_year,
                        replacement_cost
                    FROM
                        film
                    WHERE
                        replacement_cost > ANY
                        (
                            SELECT DISTINCT(replacement_cost)
                            FROM film
                            WHERE replacement_cost < 12.99
                            AND replacement_cost > 9.99
                        )
                    ORDER BY
                            title ASC;
                ''').tail(10)

,title,release_year,replacement_cost
900,WORKER TARZAN,2006,26.99
901,WORKING MICROCOSMOS,2006,22.99
902,WORLD LEATHERNECKS,2006,13.99
903,WORST BANGER,2006,26.99
904,WRATH MILE,2006,17.99
905,WYOMING STORM,2006,29.99
906,YENTL IDAHO,2006,11.99
907,YOUTH KICK,2006,14.99
908,ZOOLANDER FICTION,2006,28.99
909,ZORRO ARK,2006,18.99


This query is saying, if the film `replacement_cost` is greater than `ANY` of the results of the subquery, then return the movie. So that's all movies with `replacement_cost > 10.99 or 11.99` which equates to `replacement_cost >= 11.99`.

> Just a small note about `IN` and `ANY`, if we had stated `replacement_cost = ANY` then this would be logically equivalent to `replacement_cost IN` and can be used interchangeably. Both statements will be checking the current row for equality against values in the subquery. Though the use of `ANY` does give you use of other operators making it the more versatile option.

#### `NOT IN`

In [36]:
db.execute_query('''
                        SELECT title,
                                release_year,
                                replacement_cost
                        FROM
                                film
                        WHERE
                                replacement_cost NOT IN 
                                (
                                        SELECT DISTINCT(replacement_cost)
                                        FROM film
                                        WHERE replacement_cost < 12.99
                                        AND replacement_cost > 9.99
                                )
                        ORDER BY
                                title ASC
                ''').tail(10)
# ADDED THE 'ORDER BY' for comparing results against other queries

,title,release_year,replacement_cost
892,WORKER TARZAN,2006,26.99
893,WORKING MICROCOSMOS,2006,22.99
894,WORLD LEATHERNECKS,2006,13.99
895,WORST BANGER,2006,26.99
896,WRATH MILE,2006,17.99
897,WYOMING STORM,2006,29.99
898,YOUNG LANGUAGE,2006,9.99
899,YOUTH KICK,2006,14.99
900,ZOOLANDER FICTION,2006,28.99
901,ZORRO ARK,2006,18.99


This returns all movies where their replacement_cost is not one of `10.99`, `11.99`. So would return all rows where `replacement_cost < 10.99` and `replacement_cost > 11.99`.

#### `<= ALL`

In [40]:
db.execute_query('''
					SELECT title,
						release_year,
						replacement_cost
					FROM
						film
					WHERE
						replacement_cost <= ALL
						(
							SELECT DISTINCT(replacement_cost)
							FROM film
							WHERE replacement_cost < 12.99
							AND replacement_cost > 9.99
						)
					ORDER BY
							title ASC;
                ''').tail(10)

,title,release_year,replacement_cost
80,TRUMAN CRAZY,2006,9.99
81,TURN STAR,2006,10.99
82,VALENTINE VANISHING,2006,9.99
83,VIDEOTAPE ARSENIC,2006,10.99
84,WAIT CIDER,2006,9.99
85,WARLOCK WEREWOLF,2006,10.99
86,WITCHES PANIC,2006,10.99
87,WRONG BEHAVIOR,2006,10.99
88,YOUNG LANGUAGE,2006,9.99
89,ZHIVAGO CORE,2006,10.99


The `ALL` keyword is checking that the current row is less or equal to **all** of `10.99 and 11.99`. The query would then return all movies with `replacement_cost <= 10.99`. Why `10.99`?  Remember with the `ALL` keyword we need the `replacement_cost` condition to be `True` for **all** conditions for the row to be returned.

It's clear this condition won't return values `> 11.99`. The rows which the query returns have a `replacement_cost` value of `10.99` and `9.99`. Let's see why this is true by checking the conditions:

- For `11.99`:<br>
    `11.99 <= 11.99 = true` and `11.99 <= 10.99 = false` so `11.99` is not returned, since we get `False` in one case


- For `10.99`:<br>
   `10.99 <= 11.99 = true` and `10.99 <= 10.99 = true` so the `10.99` value is returned, since we got `True` for all
    
- For `9.99`:<br>
    `9.99 <= 11.99 = true` and `9.99 <= 10.99 = true` which is also returned, since it is `True` for all

### <h3 style="color: rgb(241, 90, 36)"> Multiple column subqueries</h3>

*Multiple column subqueries* are named as such because the **subquery** returns multiple columns. The query is usually used to filter or sort data in the **outer query** based on the results returned by the **subquery**.

An example of a **multiple column subquery** which retrieves all films with the maximum `rental_cost` and `rental_duration`:

In [ ]:
SELECT title,
       rental_duration,
       rental_rate
FROM
    film
WHERE
    (rental_duration, rental_rate) IN
    (
        SELECT MAX(rental_duration),
               MAX(rental_rate)
        FROM
            film
    );

The subquery returns the results:

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/14.%20Types%20of%20Subqueries/images/maximum_rental_rate.png?modified=232&raw=1'>

Since our **outer query** is checking that the `rental_rate` and `rental_duration` exist within the **subquery** results, it will return all films which match those values. Which is to say the outer query is returning all results where the `rental_duration` and `rental_rate` are both at their maximum.

#### <h3 style="color: rgb(241, 90, 36)"> Correlated subqueries</h3>

*Correlated subqueries* are named as such, since the output of the **subquery** depends on the results form the **outside query**, hence the two queries are **correlated**. A **correlated subquery** refers to a column which is not in the table of its `FROM` clause. These subqueries are used within the `SELECT`, `WHERE`, `HAVING` and `FROM` clauses. Let's take a look at an example of using a **correlated query** in the `SELECT` statement:

In [45]:
db.execute_query('''
                    SELECT customer_id,
                        first_name,
                        last_name,
                        (
                                SELECT ROUND(AVG(amount), 2) AS average_customer_payment
                                FROM payment
                                WHERE customer.customer_id = payment.customer_id            # this line is what makes a subquery by referencing a table no in its FROM clause but in the outer query
                            )
                    FROM
                        customer;
                ''')

,customer_id,first_name,last_name,average_customer_payment
0,1,MARY,SMITH,3.71
1,2,PATRICIA,JOHNSON,4.77
2,3,LINDA,WILLIAMS,5.22
3,4,BARBARA,JONES,3.72
4,5,ELIZABETH,BROWN,3.81
...,...,...,...,...
594,595,TERRENCE,GUNDERSON,3.92
595,596,ENRIQUE,FORSYTHE,3.45
596,597,FREDDIE,DUGGAN,3.99
597,598,WADE,DELVALLE,3.81


In the `SELECT` statement we've used the subquery to return the average payment made by each customer and included this as a column in our results. We've used the subquery to get the average `amount` paid by each customer. Notice that in the `WHERE` statement of the subquery we've matched the columns on the `customer_id` from `payment` and `customer` columns. So, the **inner query** is referencing the **outer query**, this creates the **correlation** between the two tables.

We could have performed the same query using a `JOIN`:

In [43]:
db.execute_query('''
                    SELECT customer.customer_id,
                        first_name,
                        last_name,
                        ROUND(AVG(payment.amount), 2) AS average_customer_payment
                    FROM
                        customer
                    INNER JOIN
                        payment ON payment.customer_id = customer.customer_id
                    GROUP BY
                        customer.customer_id, first_name, last_name
                    ORDER BY
                        customer_id;
                ''')

,customer_id,first_name,last_name,average_customer_payment
0,1,MARY,SMITH,3.71
1,2,PATRICIA,JOHNSON,4.77
2,3,LINDA,WILLIAMS,5.22
3,4,BARBARA,JONES,3.72
4,5,ELIZABETH,BROWN,3.81
...,...,...,...,...
594,595,TERRENCE,GUNDERSON,3.92
595,596,ENRIQUE,FORSYTHE,3.45
596,597,FREDDIE,DUGGAN,3.99
597,598,WADE,DELVALLE,3.81


With **correlated queries** you can often get the same information using a `JOIN`. In most cases a `JOIN` is more efficient, as subqueries are executed on every row of the table whereas a `JOIN` is performed once. It is rare a subquery outperforms a `JOIN` so when given the choice, try and write the statement with a `JOIN`.

#### <h3 style="color: rgb(241, 90, 36)"> Nested subqueries</h3>
*Nested subqueries* are queries which are **nested** inside other subqueries. Remember that queries are executed inside out, so if you have a subquery inside another subquery, the innermost query will be executed first:

In [46]:
db.execute_query('''
					SELECT title,
						description
					FROM
						film
					WHERE
						film_id IN
						(
							SELECT film_id
							FROM film_actor
							WHERE actor_id IN
							(
								SELECT actor_id
								FROM actor
								WHERE first_name = 'CHRISTIAN'
							)
						);
                ''')

,title,description
0,OPPOSITE NECKLACE,A Fateful Epistle of a Crocodile And a Moose w...
1,JAWBREAKER BROOKLYN,A Stunning Reflection of a Boat And a Pastry C...
2,BETRAYED REAR,A Emotional Character Study of a Boat And a Pi...
3,PRINCESS GIANT,A Thrilling Yarn of a Pastry Chef And a Monkey...
4,VACATION BOONDOCK,A Fanciful Character Study of a Secret Agent A...
...,...,...
71,FELLOWSHIP AUTUMN,A Lacklusture Reflection of a Dentist And a Hu...
72,OKLAHOMA JUMANJI,A Thoughtful Drama of a Dentist And a Womanize...
73,GOLDFINGER SENSIBILITY,A Insightful Drama of a Mad Scientist And a Hu...
74,SPIRITED CASUALTIES,A Taut Story of a Waitress And a Man who must ...


Let's take a look at what this **nested query** is performing, the **innermost query** is executed first:

In [ ]:
SELECT actor_id
FROM actor
WHERE first_name = 'CHRISTIAN'

Returning:

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/14.%20Types%20of%20Subqueries/images/nested_christian_inner.png?modified=2232&raw=1'>

The outside query then becomes:

In [ ]:
SELECT film_id
FROM film_actor
WHERE actor_id IN (10, 58, 61)
ORDER BY actor_id ASC;

Which returns the results(only partial results are shown):

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/14.%20Types%20of%20Subqueries/images/nested_outer_film_id.png?modified=32423&raw=1'>

Then finally the **outer most query** results in:

In [ ]:
SELECT title,
       description
FROM
    film
WHERE
	film_id IN (1, 9, 191, 236, ....)

This returns all movies where the actors `first_name` is `CHRISTIAN`.

#### <h3 style="color: rgb(241, 90, 36)"> Derived tables</h3>

A *derived table* is a table which is derived from an expression in the `FROM` clause of a SQL query. **Derived tables** can be useful for applying SQL logic to a **subset** or **aggregations** of a table. One important thing to remember when creating **derived tables** is, they need to have an aliases, such that they can be referenced in the outside query. An example of a query using a **derived table**:

In [48]:
db.execute_query('''
                    SELECT ROUND(AVG(payment_sums.sum_of_payments), 2) AS average_sum_of_payments
                    FROM
                        (
                            SELECT SUM(amount) AS sum_of_payments
                            FROM
                                payment
                            GROUP BY
                                customer_id
                        ) AS payment_sums;
                ''')

,average_sum_of_payments
0,112.55


Here the subquery derives a table from the `payments` table, grouping all the `customer_id`'s and summing all their payments. Notice the subquery table is also aliased as `payment_sums`, which the **outside query** can then reference to get the average of all customer payments.

If we evaluate the **inside query** and add the `customer_id` column to it, it's clear it's summing up the total payments for all customers:

In [ ]:

SELECT SUM(amount) AS sum_of_payments,
       customer_id
FROM
    payment
GROUP BY
    customer_id;

Which returns the results:

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/14.%20Types%20of%20Subqueries/images/sum_of_all_payments.png?modified=2332&raw=1'>

We then get the average and `ROUND` the results by referencing the table in the `SELECT` statement, `ROUND(AVG(payment_sums.sum_of_payments), 2)` finalising the query:

<img src='https://github.com/AI-Core/Content-Public/blob/main/Content/units/Data-Handling/3.%20SQL/14.%20Types%20of%20Subqueries/images/derived_table_results.png?modified=3234&raw=1'>

## Key Takeaways

- You can often get the same information using a `JOIN` instead of a **correlated subquery**
- When deciding which subquery you need it can be helpful to think of what information you need to return. Is it a scalar value, multiple rows?
- Using a **multiple row subquery** gives you access to the powerful `ANY`, `ALL`, `NOT IN` and `IN` operators
- Using a subquery to create a **derived table** can be a quick way to begin performing a query on a subset of data
- Continually nesting subqueries can be a great way to target incredibly specific data


